![](CRISP_DM.png)

In [ ]:
#Importing require packages
import pandas as pd
import numpy as np
from datetime import datetime
# for plotting
%matplotlib inline
pd.set_option('display.max_rows', 500)

## 20.i Preparation of Data
* Aim: To understand the final data structure
* Requirement: To be supportable each step by visual analytics

### 20.i.a Johns Hopkins GITHUB csv data

In [ ]:
# define the path of dataset file and read it using pandas
data_path=('../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
pd_raw = pd.read_csv(data_path)
pd_raw.head()

In [ ]:
#Set all date columns as time index
time_index=pd_raw.columns[4:]

In [ ]:
#Time index to dataframe as column named 'date'
df_plot = pd.DataFrame({'date':time_index})
df_plot.head()

In [ ]:
# First 5 countries in dataset
pd_raw['Country/Region'].head()

In [ ]:
# Data of germany for first 4 dates after summing of cases in relative state  
pd_raw[pd_raw['Country/Region']=='Germany'].iloc[:,4::].sum(axis=0)[0:4]

In [ ]:
#country list of future use
country_list=['Brazil','US','Spain','Germany','Italy'] 

In [ ]:
#Applying sum of infect cases for country list
for each in country_list:
    df_plot[each]=np.array(pd_raw[pd_raw['Country/Region']==each].iloc[:,4::].sum(axis=0))

In [ ]:
#Plotting
df_plot.set_index('date').plot()

### 20.i.b Data Type  for Date values in dataset

In [ ]:
# convert date columns value to datetime
t_index=[datetime.strptime( each,"%m/%d/%y") for each in df_plot.date] 
# convert back to date ISO norm in the form of string
t_str=[each.strftime('%Y-%m-%d') for each in t_index] 

In [ ]:
# Set date column as index of dataframe
df_plot['date']=t_index

type(df_plot['date'])

In [ ]:
df_plot.head()

In [ ]:
#save corrected above dataframe into our local path in CSV format
df_plot.to_csv('../data/processed/COVID_small_flat_table.csv',sep=';',index=False)

In [ ]:
df_plot.head()

## 20.ii Relational data model 

* Main features are:
    *  It must contain a unique value for each row of data.
    *  It cannot contain null values.

In [ ]:
# load file path where dataset is located on local drive
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
#Reading dataframe
pd_raw=pd.read_csv(data_path)
pd_raw.head()

In [ ]:
# Rename Country/Region and Province/State column as country and state respectively
pd_data_base=pd_raw.rename(columns={'Country/Region':'country','Province/State':'state'})
# If state data is not available place "no" 
pd_data_base['state']=pd_data_base['state'].fillna('no') 

In [ ]:
# Remove lat and long data
pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)
pd_data_base.head()

In [ ]:
# New dataframe by making changes of below
pd_rel_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},)
pd_rel_model.head()

In [ ]:
# Check data types of each columns
pd_rel_model.dtypes

In [ ]:
# set "date" as "datetime64[ns]
pd_rel_model['date']=pd_rel_model.date.astype('datetime64[ns]')

In [ ]:
pd_rel_model.dtypes

In [ ]:
# set confirmed column datatype as integer
pd_rel_model.confirmed=pd_rel_model.confirmed.astype(int)
# save dataframe to local drive in CSV format
pd_rel_model.to_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
pd_rel_model.head()

## 20.iii Relational dataframe for Johns Hopkins dataset containing state information for US

In [ ]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
pd_raw_US=pd.read_csv(data_path)
pd_raw_US.head()

In [ ]:
# dropping all non relevant columns 
pd_raw_US=pd_raw_US.drop(['UID', 'iso2', 'iso3', 'code3', 'Country_Region','FIPS', 'Admin2',  'Lat', 'Long_', 'Combined_Key'],axis=1)
# rename state columns
pd_data_base_US=pd_raw_US.rename(columns={'Province_State':'state'}).copy()

In [ ]:
## repeat the process as earlier 
pd_rel_model_US=pd_data_base_US.set_index(['state']) \
                                .T                              \
                                .stack()             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )

# set each column country value to US
pd_rel_model_US['country']='US'
# convert to datetime
pd_rel_model_US['date']=[datetime.strptime( each,"%m/%d/%y") for each in pd_rel_model_US.date] 
pd_rel_model_US.head()

In [ ]:
# update the relational file of US
pd_rel_model_all=pd_rel_model[pd_rel_model['country']!='US'].reset_index(drop=True)

In [ ]:
pd_rel_model_all=pd.concat([pd_rel_model_all,pd_rel_model_US],ignore_index=True)

In [ ]:
pd_rel_model_all.to_csv('../data/processed/20200424_COVID_relational_confirmed.csv',sep=';',index=False)

In [ ]:
print(pd_rel_model_all[pd_rel_model_all['country']=='US'])

#### 20.iv.a Verification of full sum US data

In [ ]:
ver_check_US=pd_rel_model_all[pd_rel_model_all['country']=='US']

In [ ]:
ver_check_US[['date','country','confirmed']].groupby(['date','country']).sum()